In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
tf.keras.backend.clear_session()

In [9]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [10]:
subject = 'Rio Grande Do Norte - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [11]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RN.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),...,Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio Grande Do Norte - Consumo de Cimento (t)
0,2008,443.220730,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,0.817253,0.651662,8.054622,5.483335e+08,...,0.667512,0.709150,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682,705.791000
1,2009,467.794411,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,0.819840,0.655238,8.084471,5.302764e+08,...,0.669561,0.711755,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049,694.478000
2,2010,615.433124,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,0.821734,0.658498,8.047148,5.046425e+08,...,0.671269,0.713945,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629,850.190000
3,2011,928.296392,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,0.822949,0.661438,8.031985,4.854622e+08,...,0.672647,0.715731,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734,836.975000
4,2012,1212.359182,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,0.823515,0.663198,12.625418,4.867661e+08,...,0.673384,0.716652,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622,961.431000
5,2013,1332.847938,3.658711e+07,2.061661e+06,9.253188,2.982187e+07,0.823225,0.667048,12.889274,5.225848e+08,...,0.674666,0.718456,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194,954.170000
6,2014,1327.285781,3.698650e+07,2.024778e+06,9.243102,2.982142e+07,0.822577,0.669135,13.251300,5.676413e+08,...,0.675055,0.719161,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147,886.165714
7,2015,1263.490912,3.730024e+07,1.995627e+06,9.238580,2.982594e+07,0.821459,0.669230,13.501882,5.179392e+08,...,0.674976,0.718841,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726,802.508571
8,2016,1117.745802,3.749595e+07,1.966808e+06,9.235352,2.982092e+07,0.834707,0.679146,13.466118,4.524144e+08,...,0.683189,0.729101,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474,730.261714
9,2017,983.861706,3.759092e+07,1.955499e+06,9.243656,2.984006e+07,0.833862,0.679432,13.134883,4.213689e+08,...,0.682843,0.728837,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241,646.482000


In [12]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,443.220730,3.355987e+07,2.079113e+06,9.184161,2.924771e+07,0.817253,0.651662,8.054622,5.483335e+08,2.546644e+09,...,11.760983,0.667512,0.709150,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682
1,467.794411,3.376729e+07,2.091124e+06,9.172749,2.923115e+07,0.819840,0.655238,8.084471,5.302764e+08,2.679945e+09,...,9.472585,0.669561,0.711755,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049
2,615.433124,3.462025e+07,2.094395e+06,9.216787,2.947144e+07,0.821734,0.658498,8.047148,5.046425e+08,2.891930e+09,...,9.332485,0.671269,0.713945,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629
3,928.296392,3.538151e+07,2.094876e+06,9.249340,2.964338e+07,0.822949,0.661438,8.031985,4.854622e+08,3.091988e+09,...,10.985689,0.672647,0.715731,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734
4,1212.359182,3.604796e+07,2.083213e+06,9.259759,2.976380e+07,0.823515,0.663198,12.625418,4.867661e+08,3.189506e+09,...,8.159014,0.673384,0.716652,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622
5,1332.847938,3.658711e+07,2.061661e+06,9.253188,2.982187e+07,0.823225,0.667048,12.889274,5.225848e+08,3.093872e+09,...,7.913561,0.674666,0.718456,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194
6,1327.285781,3.698650e+07,2.024778e+06,9.243102,2.982142e+07,0.822577,0.669135,13.251300,5.676413e+08,2.779340e+09,...,10.350442,0.675055,0.719161,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147
7,1263.490912,3.730024e+07,1.995627e+06,9.238580,2.982594e+07,0.821459,0.669230,13.501882,5.179392e+08,2.561731e+09,...,12.454398,0.674976,0.718841,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726
8,1117.745802,3.749595e+07,1.966808e+06,9.235352,2.982092e+07,0.834707,0.679146,13.466118,4.524144e+08,2.516859e+09,...,13.111908,0.683189,0.729101,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474
9,983.861706,3.759092e+07,1.955499e+06,9.243656,2.984006e+07,0.833862,0.679432,13.134883,4.213689e+08,2.497710e+09,...,9.479812,0.682843,0.728837,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     694.478000
1     850.190000
2     836.975000
3     961.431000
4     954.170000
5     886.165714
6     802.508571
7     730.261714
8     646.482000
9     485.012000
10    563.293000
11    744.863000
12    811.860000
13    866.416000
14           NaN
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [14]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.745108,-1.831952,0.851916,-1.776864,-1.942366,-1.428923,-1.679355,-1.408085,1.204752,-0.990234,...,0.984445,-1.525355,-1.569868,2.457245,-1.212494,1.862951,-0.705394,-0.762192,-0.756764,1.972247
1,-1.663415,-1.686476,1.061967,-2.221110,-2.019038,-0.974260,-1.304896,-1.395638,0.820657,-0.546688,...,-0.051710,-1.159167,-1.188723,0.144226,-1.582145,1.855564,-0.347758,-0.699954,-0.781667,1.489960
2,-1.172604,-1.088236,1.119167,-0.506765,-0.906710,-0.641303,-0.963539,-1.411202,0.275392,0.158672,...,-0.115145,-0.853862,-0.868295,-0.568350,-1.518190,1.208644,-1.439934,-1.053053,-0.893344,1.034020
3,-0.132519,-0.554313,1.127579,0.760462,-0.110759,-0.427758,-0.655663,-1.417525,-0.132596,0.824346,...,0.633403,-0.607606,-0.606843,-0.695505,-0.774675,0.248067,-1.287958,-1.257539,-1.198046,0.513803
4,0.811822,-0.086886,0.923630,1.166077,0.446668,-0.328271,-0.471367,0.497973,-0.104860,1.148829,...,-0.646476,-0.475880,-0.472170,0.071997,-0.284967,-0.279787,-0.574331,-1.010638,-1.104686,0.113084
5,1.212375,0.291258,0.546720,0.910252,0.715452,-0.379303,-0.068230,0.608003,0.657046,0.830615,...,-0.757614,-0.246760,-0.208132,1.159926,0.371328,-0.502457,-0.529965,-0.573928,-0.578367,-0.299977
6,1.193884,0.571377,-0.098301,0.517622,0.713363,-0.493169,0.150343,0.758971,1.615453,-0.215960,...,0.345772,-0.177266,-0.105014,0.479702,1.187601,-0.630330,0.031926,-0.036991,-0.079642,-0.497452
7,0.981804,0.791421,-0.608079,0.341588,0.734301,-0.689751,0.160247,0.863466,0.558230,-0.940033,...,1.298413,-0.191340,-0.151836,-0.445543,1.261073,-0.699772,0.993697,0.483365,0.289839,-0.481495
8,0.497288,0.928688,-1.112071,0.215918,0.711070,1.638563,1.198697,0.848551,-0.835563,-1.089341,...,1.596124,1.276482,1.349626,-1.105884,0.913840,-0.740852,0.416771,0.706397,0.759507,-0.565156
9,0.052203,0.995293,-1.309848,0.539207,0.799679,1.490090,1.228649,0.710424,-1.495939,-1.153056,...,-0.048438,1.214568,1.311056,-1.310433,0.799449,-0.768068,0.215438,1.012997,1.226895,-0.874702


In [15]:
reshaped_train_input = dfToInputRNN(train_input)

In [16]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.06447243,  0.74079602, -1.11662735, -0.15231429,
          0.31644791,  0.95022742,  1.07096768,  0.63536962,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  1

In [17]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     694.478000
1     850.190000
2     836.975000
3     961.431000
4     954.170000
5     886.165714
6     802.508571
7     730.261714
8     646.482000
9     485.012000
10    563.293000
11    744.863000
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [18]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    744.863
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [19]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.424256,0.37406,-0.9372,-1.199219,-0.288358,0.55467,0.973144,0.602486,1.108405,2.635823,...,-2.260447,1.096579,0.927289,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


In [20]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.42425559,  0.37406027, -0.93719981, -1.19921903,
         -0.28835757,  0.55466994,  0.9731445 ,  0.60248617,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  1

In [21]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    811.86
Name: Rio Grande Do Norte - Consumo de Cimento (t), dtype: float64

In [33]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [34]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(200)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [35]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[143843968, 1177703209, 2751554996, 3277521246, 1288444793, 590227426, 942151365, 2231371318, 829937526, 3888246389, 967504365, 2933835534, 3203126961, 1125888971, 1078464083, 912075205, 3894031476, 3972207234, 3706514706, 2279831235, 1437336545, 1906892128, 2103645459, 3448238571, 326609405, 3701791237, 1733302028, 3538366508, 2510622690, 3101426898, 635449662, 2241957385, 4090702219, 1500229679, 2356676779, 3789559033, 3457891614, 3546948038, 1353799417, 1189757756, 748593489, 2859232390, 2922013220, 799734400, 4070394592, 1963147140, 3431992659, 4227948873, 504867795, 3277662947, 1213830203, 1011078192, 2723295628, 3615334631, 1360019195, 2061817560, 2476256936, 3107895484, 2263789812, 3761538114, 31659508, 1539543814, 1584714456, 2213520843, 1577049743, 3507205452, 1535816992, 3757894379, 3300402375, 616010421, 2106036355, 201080107, 449743128, 1480288178, 3799171962, 294866356, 428146535, 1390859371, 3648131253, 2964677480, 2787192564, 3610075441, 33049154, 4057550948, 3365761939,

loss: 137.2224578857422


Step: 73 ___________________________________________
loss: 230.49786376953125


Step: 74 ___________________________________________
loss: 237.7100830078125


Step: 75 ___________________________________________
loss: 136.12547302246094


Step: 76 ___________________________________________
loss: 368.630615234375


Step: 77 ___________________________________________
loss: 265.9064025878906


Step: 78 ___________________________________________
loss: 173.962890625


Step: 79 ___________________________________________
loss: 264.2769775390625


Step: 80 ___________________________________________
loss: 176.7780303955078


Step: 81 ___________________________________________
loss: 461.8192138671875


Step: 82 ___________________________________________
loss: 357.6060791015625


Step: 83 ___________________________________________
loss: 378.4449157714844


Step: 84 ___________________________________________
loss: 330.7549133300781


Step: 85 ____________________

loss: 427.8349609375


Step: 176 ___________________________________________
loss: 371.6112060546875


Step: 177 ___________________________________________
loss: 360.6710510253906


Step: 178 ___________________________________________
loss: 213.772216796875


Step: 179 ___________________________________________
loss: 324.3786315917969


Step: 180 ___________________________________________
loss: 154.5428924560547


Step: 181 ___________________________________________
loss: 237.69297790527344


Step: 182 ___________________________________________
loss: 221.91355895996094


Step: 183 ___________________________________________
loss: 276.9837646484375


Step: 184 ___________________________________________
loss: 414.9894104003906


Step: 185 ___________________________________________
loss: 72.31890869140625
winner_seed: 3722537537


Step: 186 ___________________________________________
loss: 286.8150329589844


Step: 187 ___________________________________________
loss: 189.49107360

In [36]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 602353.3125
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 588776.0625
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 545186.9375
Epoch 4/10000
1/1 [==============================] - 0s 3ms/step - loss: 254615.6719
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 36522.7070
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 26379.8223
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 9490.2217
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 8578.2920
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 10142.7305
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 7035.9897
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 8137.3716
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 8

1/1 [==============================] - 0s 3ms/step - loss: 5701.4102
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 4652.6733
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 6801.2319
Epoch 101/10000
1/1 [==============================] - 0s 3ms/step - loss: 5442.9858
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 5406.4487
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 3794.1624
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 7857.4976
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 4069.5000
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 5298.9756
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 6343.3223
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 5060.7563
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 4453.27

1/1 [==============================] - 0s 4ms/step - loss: 3593.9500
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 2858.0862
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 4523.5308
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 3495.8542
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 4100.4321
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 1845.0016
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 3403.8674
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 628.1428
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 1845.2455
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 1702.5385
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 1753.0864
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 2721.95

1/1 [==============================] - 0s 4ms/step - loss: 4692.7891
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 1909.1270
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 4004.8469
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 2086.5696
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 4961.2617
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 4474.5718
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 6183.9204
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 1907.6753
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 2317.0156
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 3824.9114
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 2475.8723
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 2456.2

1/1 [==============================] - 0s 4ms/step - loss: 1203.4200
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 2920.0605
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 3187.3203
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 2305.2129
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 2146.2500
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 2648.0886
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 1643.9178
Epoch 393/10000
1/1 [==============================] - 0s 3ms/step - loss: 2041.2823
Epoch 394/10000
1/1 [==============================] - 0s 3ms/step - loss: 2292.3040
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 2306.2451
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 3508.9756
Epoch 397/10000
1/1 [==============================] - 0s 3ms/step - loss: 2720.9

...

1/1 [==============================] - 0s 4ms/step - loss: 811.9724
Epoch 1344/10000
1/1 [==============================] - 0s 4ms/step - loss: 1179.1224
Epoch 1345/10000
1/1 [==============================] - 0s 3ms/step - loss: 1257.7775
Epoch 1346/10000
1/1 [==============================] - 0s 4ms/step - loss: 2065.6348
Epoch 1347/10000
1/1 [==============================] - 0s 4ms/step - loss: 1814.9203
Epoch 1348/10000
1/1 [==============================] - 0s 4ms/step - loss: 1660.2523
Epoch 1349/10000
1/1 [==============================] - 0s 4ms/step - loss: 933.8540
Epoch 1350/10000
1/1 [==============================] - 0s 4ms/step - loss: 4350.1704
Epoch 1351/10000
1/1 [==============================] - 0s 4ms/step - loss: 2608.9375
Epoch 1352/10000
1/1 [==============================] - 0s 4ms/step - loss: 961.5312
Epoch 1353/10000
1/1 [==============================] - 0s 4ms/step - loss: 2146.0276
Epoch 1354/10000
1/1 [==============================] - 0s 4ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 1937.5541
Epoch 1439/10000
1/1 [==============================] - 0s 4ms/step - loss: 1649.8022
Epoch 1440/10000
1/1 [==============================] - 0s 4ms/step - loss: 1810.9586
Epoch 1441/10000
1/1 [==============================] - 0s 4ms/step - loss: 1448.9448
Epoch 1442/10000
1/1 [==============================] - 0s 4ms/step - loss: 1183.1155
Epoch 1443/10000
1/1 [==============================] - 0s 4ms/step - loss: 1960.0514
Epoch 1444/10000
1/1 [==============================] - 0s 4ms/step - loss: 1537.0732
Epoch 1445/10000
1/1 [==============================] - 0s 4ms/step - loss: 3248.4597
Epoch 1446/10000
1/1 [==============================] - 0s 4ms/step - loss: 1924.5157
Epoch 1447/10000
1/1 [==============================] - 0s 4ms/step - loss: 2604.6086
Epoch 1448/10000
1/1 [==============================] - 0s 4ms/step - loss: 1121.9360
Epoch 1449/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 1399.8175
Epoch 1534/10000
1/1 [==============================] - 0s 5ms/step - loss: 1135.6857
Epoch 1535/10000
1/1 [==============================] - 0s 4ms/step - loss: 1650.8672
Epoch 1536/10000
1/1 [==============================] - 0s 4ms/step - loss: 763.5402
Epoch 1537/10000
1/1 [==============================] - 0s 4ms/step - loss: 1864.8531
Epoch 1538/10000
1/1 [==============================] - 0s 4ms/step - loss: 1091.2401
Epoch 1539/10000
1/1 [==============================] - 0s 4ms/step - loss: 2037.8346
Epoch 1540/10000
1/1 [==============================] - 0s 5ms/step - loss: 1451.5057
Epoch 1541/10000
1/1 [==============================] - 0s 4ms/step - loss: 2967.3684
Epoch 1542/10000
1/1 [==============================] - 0s 4ms/step - loss: 2831.5657
Epoch 1543/10000
1/1 [==============================] - 0s 4ms/step - loss: 1982.9332
Epoch 1544/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 5ms/step - loss: 2458.9290
Epoch 1629/10000
1/1 [==============================] - 0s 4ms/step - loss: 1924.4054
Epoch 1630/10000
1/1 [==============================] - 0s 5ms/step - loss: 1032.5059
Epoch 1631/10000
1/1 [==============================] - 0s 5ms/step - loss: 1245.3673
Epoch 1632/10000
1/1 [==============================] - 0s 4ms/step - loss: 1817.9572
Epoch 1633/10000
1/1 [==============================] - 0s 4ms/step - loss: 715.0726
Epoch 1634/10000
1/1 [==============================] - 0s 4ms/step - loss: 1821.4166
Epoch 1635/10000
1/1 [==============================] - 0s 4ms/step - loss: 1407.9652
Epoch 1636/10000
1/1 [==============================] - 0s 4ms/step - loss: 1732.4606
Epoch 1637/10000
1/1 [==============================] - 0s 4ms/step - loss: 1203.0048
Epoch 1638/10000
1/1 [==============================] - 0s 4ms/step - loss: 1032.2795
Epoch 1639/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 6ms/step - loss: 1672.1021
Epoch 1724/10000
1/1 [==============================] - 0s 5ms/step - loss: 2127.0215
Epoch 1725/10000
1/1 [==============================] - 0s 4ms/step - loss: 1376.5475
Epoch 1726/10000
1/1 [==============================] - 0s 6ms/step - loss: 2376.3386
Epoch 1727/10000
1/1 [==============================] - 0s 4ms/step - loss: 402.4809
Epoch 1728/10000
1/1 [==============================] - 0s 4ms/step - loss: 1860.4917
Epoch 1729/10000
1/1 [==============================] - 0s 4ms/step - loss: 1346.0709
Epoch 1730/10000
1/1 [==============================] - 0s 4ms/step - loss: 581.7153
Epoch 1731/10000
1/1 [==============================] - 0s 5ms/step - loss: 1997.2836
Epoch 1732/10000
1/1 [==============================] - 0s 6ms/step - loss: 1454.4330
Epoch 1733/10000
1/1 [==============================] - 0s 5ms/step - loss: 1164.4222
Epoch 1734/10000
1/1 [==============================] - 0s 4ms/step - los

In [37]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 575ms/step
[811.86](test_target) - [731.86975](prediction) = 79.99024902343751


In [38]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.09852714633488226

In [39]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [40]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.862482,-0.920976,-1.384697,-0.378948,-0.630183,-1.282734,-1.243181,-0.462543,1.148205,-1.122043,...,1.412346,-1.260035,-1.258506,1.377871,1.395565,0.719114,0.276338,0.495192,0.906319,1.236047
1,-0.539374,-0.468948,0.443432,-0.990483,-0.781336,0.125669,0.037745,1.388657,0.140893,-0.184474,...,-0.643250,0.073929,0.070575,-0.413067,-0.896020,0.695031,1.062967,0.899613,0.487021,-0.022926
2,1.401856,1.389925,0.941266,1.369431,1.411518,1.157065,1.205436,-0.926114,-1.289098,1.306518,...,-0.769096,1.186106,1.187932,-0.964804,-0.499545,-1.414145,-1.339305,-1.394805,-1.393340,-1.213121


train_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
0,694.478
1,850.190
2,836.975


test_input:


...

,Rio Grande Do Norte - Produção de Cimento (t),Rio Grande do Norte - PIB - Estadual,Rio Grande do Norte - PIB - Construção Civil,Rio Grande do Norte - PIB - Per Capita,Rio Grande do Norte - PIB - Preços de Mercado,Rio Grande do Norte - IDH Longevidade,Rio Grande do Norte - IDH Educacao,Rio Grande do Norte - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Rio Grande do Norte - IDH Renda,Rio Grande do Norte - IDH,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.424256,0.37406,-0.9372,-1.199219,-0.288358,0.55467,0.973144,0.602486,1.108405,2.635823,...,-2.260447,1.096579,0.927289,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


test_target:


,Rio Grande Do Norte - Consumo de Cimento (t)
12,811.86


1/1 [==============================] - 1s 546ms/step
Error: 79.99024902343751




[841.9405517578125,
 948.6455688476562,
 732.52783203125,
 816.0967407226562,
 806.3642578125,
 734.271240234375,
 645.74609375,
 502.52752685546875,
 556.6712646484375,
 731.8697509765625]

In [41]:
display(targets)
display(predictions)

[961.431,
 954.17,
 886.1657142857142,
 802.5085714285714,
 730.2617142857143,
 646.482,
 485.012,
 563.293,
 744.8629999999999,
 811.86]

[841.9405517578125,
 948.6455688476562,
 732.52783203125,
 816.0967407226562,
 806.3642578125,
 734.271240234375,
 645.74609375,
 502.52752685546875,
 556.6712646484375,
 731.8697509765625]

In [42]:
mae = mean_absolute_error(predictions, targets)
mae

94.58142659737723

In [43]:
porcentage = mae/np.mean(targets)
porcentage

0.12467814475362098